# Portkey | Building Resilient LLM Apps

**Portkey** is a full-stack LLMOps platform that productionizes your Gen AI app reliably and securely.

### Key Features of Portkey

1. **AI Gateway**:
    - **Automated Fallbacks & Retries**: Ensure your application remains functional even if a primary service fails.
    - **Load Balancing**: Efficiently distribute incoming requests among multiple models.
    - **Semantic Caching**: Reduce costs and latency by intelligently caching results.
    
2. **Observability**:
    - **Logging**: Keep track of all requests for monitoring and debugging.
    - **Requests Tracing**: Understand the journey of each request for optimization.
    - **Custom Tags**: Segment and categorize requests for better insights.

To harness these features, let's start with the setup:


In [ ]:
# Installing the Portkey AI python SDK developed by the Portkey team
!pip install portkey-ai -U
!portkey --version

In [ ]:
# Importing necessary libraries and modules
import portkey as pk
from portkey import Config, LLMOptions
from getpass import getpass

#### **Step 1: Get your Portkey API key**

Log into [Portkey here](https://app.portkey.ai/), then click on the profile icon on top right and "Copy API Key". Let's also set OpenAI & Anthropic API keys.

In [ ]:
# Enter the password on the prompt window.
API_KEY = getpass("Enter your PORTKEY_API_KEY ")

# Setting the API key
pk.api_key = API_KEY

# NOTE: For adding custom url, uncomment this line and add your custom url in a selfhosted version.
# pk.base_url = ""

#### **Step 2: Configure Portkey Features**

To harness the full potential of Portkey, you can configure various features as illustrated above. Here's a guide to all Portkey features and the expected values:

| Feature             | Config Key              | Value(Type)                                      | Required    |
|---------------------|-------------------------|--------------------------------------------------|-------------|
| API Key             | `api_key`               | `string`                                         | ✅ Required (can be set externally) |
| Mode                | `mode`                  | `fallback`, `ab_test`, `single`              | ✅ Required |
| Cache Type          | `cache_status`          | `simple`, `semantic`                             | ❔ Optional |
| Force Cache Refresh | `cache_force_refresh`   | `boolean`                              | ❔ Optional |
| Cache Age           | `cache_age`             | `integer` (in seconds)                           | ❔ Optional |
| Trace ID            | `trace_id`              | `string`                                         | ❔ Optional |
| Retries         | `retry`           | `integer` [0,5]                                  | ❔ Optional |
| Metadata            | `metadata`              | `json object` [More info](https://docs.portkey.ai/key-features/custom-metadata)          | ❔ Optional |
| Base URL | `base_url` | `url` | ❔ Optional |


To set up Portkey for different modes and features, refer to the provided IPython Notebook examples in the examples/ directory.

For more information and detailed documentation, please visit [Portkey Documentation](https://docs.portkey.ai/).

## Example-1: Configuring Portkey for Fallback Mode
In this example, we'll demonstrate how to configure Portkey for the Fallback Mode using the sdk. Fallback Mode allows you to define a backup strategy when your primary service is unavailable.

`Note`: The order of definition of LLMOptions is important for fallbacks. Ensure that you define your fallback strategy in the order of preference. This ensures that your fallback logic is in place and ready to be used when needed.

In [ ]:
pk.config = Config(
    mode="fallback",
    llms=[
        LLMOptions(
            model="text-davinci-002",
            virtual_key="open-ai-key-66a67d",
            provider="openai",
        ),
        LLMOptions(
            model="claude-2",
            virtual_key="anthropic-key-351feb",
            provider="anthropic",
            max_tokens=250,
        ),
    ],
)

In [ ]:
# Example 1: Basic example

response = pk.Completions.create(prompt="Who are you ?")

print(response.choices[0].text)

In [ ]:
# Example 3: Streaming results

response3 = pk.Completions.create(
    prompt="Translate the following English text to French: 'Hello, how are you?'",
    stream=True,  # Stream back partial progress
)

for event in response3:
    if event.choices[0].text:
        print(event.choices[0].text, end="", flush=True)

## Example-2: Configuring Portkey for Load Balancing (A/B test) Mode

To utilize Portkey's Load Balancing Mode, follow the steps below. Load Balancing Mode enables you to distribute incoming requests across multiple services to ensure high availability and scalability.

`NOTE`: Loadbalance is also called A/B test.

In [ ]:
pk.config = Config(
    mode="ab_test",
    llms=[
        LLMOptions(
            model="text-davinci-002",
            virtual_key="open-ai-key-66a67d",
            provider="openai",
            weight=0.4,
        ),
        LLMOptions(
            model="claude-2",
            virtual_key="anthropic-key-351feb",
            provider="anthropic",
            max_tokens=250,
            weight=0.6,
        ),
    ],
)

In [ ]:
# Example 1: Basic example

response = pk.Completions.create(
    prompt="Summarize the key points from the article titled 'The Impact of Climate Change on Global Biodiversity.'"
)

print(response.choices[0].text)

In [ ]:
# Example 3: Streaming results

response3 = pk.Completions.create(
    prompt="Generate a creative short story about a detective solving a mysterious case.",
    stream=True,  # Stream back partial progress
)

for event in response3:
    if event.choices[0].text is None:
        break
    print(event.choices[0].text, end="", flush=True)